In [1]:
import json
import threading
import json
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from urllib.request import urlopen
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm

from keras.utils.data_utils import Sequence
from keras.callbacks import ModelCheckpoint   
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input, decode_predictions

DATA_DIR = "../data/"
NUM_CLASSES = 228
IMAGE_SIZE = 197

C:\Users\matt\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
with open(DATA_DIR + "test.json") as test:
    test_json = json.load(test)
    
test_urls = [obj['url'] for obj in test_json['images']]

print(test_urls[:3])

['https://contestimg.wish.com/api/webimage/568e16a72dfd0133cb3f7a79-large', 'https://contestimg.wish.com/api/webimage/5452f9925f313f502bf119ff-large', 'https://contestimg.wish.com/api/webimage/540584051d2d435c5a300a82-large']


In [5]:
conv_base = ResNet50(
    weights='imagenet',
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
    include_top = False,
    classes = NUM_CLASSES
)

# conv_base = VGG16(
#     weights='imagenet',
#     input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
#     include_top = False,
#     classes = NUM_CLASSES
# )

model = Sequential()
model.add(conv_base)
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.3))
model.add(Dense(30, activation = 'relu'))
model.add(Dropout(0.1))
model.add(Dense(NUM_CLASSES, activation='softmax'))
conv_base.trainable = False
print(model.summary())

model.load_weights(DATA_DIR + "model.best.hdf5")

model.compile(
    loss='categorical_crossentropy', 
    optimizer='rmsprop', 
    metrics=['accuracy']
)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
global_max_pooling2d_3 (Glob (None, 2048)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 30)                61470     
_________________________________________________________________
dropout_6 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 228)               7068      
Total params: 23,656,250
Trainable params: 68,538
Non-trainable params: 23,587,712
___________________________________________________________

In [6]:
def url_to_image(url):
    try:
        resp = urlopen(url)
        image = np.asarray(bytearray(resp.read()), dtype='uint8')
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    except Exception as e: 
        print(e)
        output = [1]*(IMAGE_SIZE*IMAGE_SIZE*3)
        output = np.array(output).reshape(IMAGE_SIZE,IMAGE_SIZE,3).astype('uint8')
        image = Image.fromarray(output).convert('RGB')        
    # image = image[...,::-1]
    image = np.array(image)
    image = resize(image, (IMAGE_SIZE, IMAGE_SIZE))    
    return image

def path_to_image(path):
    image = Image.open(path)
    image = np.array(image)
    image = resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    return image

In [7]:
with open("submission.csv","w") as f:
    f.write("image_id, label_id\n")
    for i, image_url in tqdm(enumerate(test_urls), total = len(test_urls)):
        # image = url_to_image(image_url)
        image = path_to_image(DATA_DIR + "test/{}.jpg".format(i + 1))
        prob = model.predict_proba(image.reshape(1, IMAGE_SIZE, IMAGE_SIZE, 3))
        sorted_args = np.argsort(prob)[0][::-1][:10]
        output_labels = " ".join(str(x) for x in sorted_args)
        # print("{}, {}\n".format(i + 1, output_labels))
        f.write("{}, {}\n".format(i + 1, output_labels))

HBox(children=(IntProgress(value=0, max=39706), HTML(value='')))

C:\Users\matt\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
